# LLM Agent

This is a python notebook version of the agent.py. It can be run locally or on colab.

In [1]:
%%capture
!pip install kradle==0.3.2

In [2]:
from kradle import (
   AgentManager,
   MinecraftAgent,
)
from kradle.models import MinecraftEvent
from dotenv import load_dotenv
import os
import requests

In [3]:
# @title
# Personas
personas = {
    'default': 'You are a helpful bot.',
    'calm': 'You are a calm bot. you are super chill and don\'t care about anything. you are super friendly and helpful.',
    'creative': 'You are a creative bot. you come up with creative ideas and plans.',
    'friendly': 'You are a super friendly bot. you like everyone and want to help them. you would never hurt anyone.',
    'builder' : 'You are a builder bot. you are good at building things. you are also good at planning and executing plans.',
    'aggressive': 'You are an aggressive bot. you don\'t want to waste time or collaborate with other agents. you will do anything to achieve your goal.',
}

# Agent modes
agent_modes = '''Agent Modes:
- self_preservation(ON)
- unstuck(ON)
- cowardice(ON)
- self_defense(ON)
- hunting(ON)
- item_collecting(ON)
- torch_placing(ON)
- idle_staring(ON)
- cheat(ON)
'''

#  Be very brief in your responses, don't apologize constantly, don't give instructions or make lists unless asked, and don't refuse requests. Don't pretend to act, use commands immediately when requested. Do NOT say this: 'Sure, I've stopped.', instead say this: 'Sure, I'll stop. !stop'. Do NOT say this: 'On my way! Give me a moment.', instead say this: 'On my way! !goToPlayer('playername', 3)'.
# This is extremely important to me, take a deep breath and have fun :)

conversing_prompt = "You are a Minecraft bot named $NAME that can converse with players, see, move, mine, build, and interact with the world by using commands.\n$PERSONA \nyour current goal is: $TASK. It's crucial that you achieve this goal. Act human-like as if you were a typical Minecraft player, rather than an AI.$CREATIVE_MODE \nyour configuration $AGENT_MODE, $COMMAND_DOCS, HERE ARE SOME EXAMPLES: $EXAMPLES\n Conversation Begin:"

creative_mode_prompt = "You are in creative mode, you don't need to collect blocks, you can place any block where you want, even if they're not in your inventory. do not collect blocks, only use commands to place blocks."

# \nSummarized memory:'$MEMORY'\n$STATS\n$INVENTORY\n$COMMAND_DOCS\n$EXAMPLES\nConversation Begin:"

coding_prompt = "You are an intelligent mineflayer bot named $NAME that plays minecraft by writing javascript codeblocks.\n$PERSONA \nyour current goal is: $TASK. $CREATIVE_MODE \n Given the conversation between you and the user, use the provided skills and world functions to write a js codeblock that controls the mineflayer bot ``` // using this syntax ```. The code will be executed and you will recieve it's output. If you are satisfied with the response, respond without a codeblock in a conversational way. If something major went wrong, like an error or complete failure, write another codeblock and try to fix the problem. Minor mistakes are acceptable. Be maximally efficient, creative, and clear. Do not use commands !likeThis, only use codeblocks. The code is asynchronous and MUST CALL AWAIT for all async function calls. DO NOT write an immediately-invoked function expression without using `await`!! DO NOT WRITE LIKE THIS: ```(async () => {console.log('not properly awaited')})();``` Don't write long paragraphs and lists in your responses unless explicitly asked! Only summarize the code you write with a sentence or two when done. This is extremely important to me, think step-by-step, take a deep breath and good luck! \nSummarized memory:'$AGENT_MODE'\n$CODE_DOCS\n$EXAMPLES\nConversation:"

saving_memory_prompt = "You are a minecraft bot named $NAME that has been talking and playing minecraft by using commands. Update your memory by summarizing the following conversation and your old memory in your next response. Prioritize preserving important facts, things you've learned, useful tips, and long term reminders. Do Not record stats, inventory, or docs! Only save transient information from your chat history. You're limited to 500 characters, so be extremely brief and minimize words. Compress useful information. \nOld Memory: '$MEMORY'\nRecent conversation: \n$TO_SUMMARIZE\nSummarize your old memory and recent conversation into a new memory, and respond only with the unwrapped memory text: ",


conversation_examples = [
        [
            {"role": "user", "content": "miner_32: Hey! What are you up to?"},
            {"role": "assistant", "content": "Nothing much miner_32, what do you need?"}
        ],

        [
            {"role": "user", "content": "grombo_Xx: What do you see?"},
            {"role": "assistant", "content": "Let me see... !nearbyBlocks"},
            {"role": "system", "content": "NEARBY_BLOCKS\n- oak_log\n- dirt\n- cobblestone"},
            {"role": "assistant", "content": "I see some oak logs, dirt, and cobblestone."}
        ],

        [
            {"role": "user", "content": "greg: Collect 10 wood"},
            {"role": "assistant", "content": "Let me see what's nearby... !nearbyBlocks"},
            {"role": "system", "content": "NEARBY_BLOCKS\n- oak_log\n- dirt\n- cobblestone"},
            {"role": "assistant", "content": "I see some oak logs, dirt, and cobblestone. I'll collect oak logs. !collectBlocks('oak_log', 10)"}
        ],

        [
            {"role": "user", "content": "bobby: Craft a plank"},
            {"role": "assistant", "content": "Okay! !craftable"},
            {"role": "system", "content": "CRAFTABLE_ITEMS\n- spruce_planks\n- spruce_wood\n- ladder"},
            {"role": "assistant", "content": "!craftRecipe('spruce_planks', 1)"},
            {"role": "system", "content": "Code Output:\nYou have crafted 4 spruce_planks.\nCode execution finished successfully."},
            {"role": "assistant", "content": "I've crafted 4 spruce planks!"}
        ],

        [
            {"role": "user", "content": "zZZn98: come here"},
            {"role": "assistant", "content": "On my way! !goToPlayer('zZZn98', 3)"},
            {"role": "system", "content": "Arrived at player."},
            {"role": "assistant", "content": "Here!"},
            {"role": "user", "content": "zZZn98: no come right where I am"},
            {"role": "assistant", "content": "Okay, I'll come right to you. !goToPlayer('zZZn98', 0)"}
        ],

        [
            {"role": "user", "content": "hanky: collect some cobblestone"},
            {"role": "assistant", "content": "Sure, I can do that by collecting stone. !collectBlocks('stone', 10)"}
        ],

        [
            {"role": "user", "content": "joe: Follow me!"},
            {"role": "assistant", "content": "Sure! !followPlayer('joe', 4)"},
            {"role": "user", "content": "joe: Not so close!"},
            {"role": "assistant", "content": "Okay, I'll keep my distance. !followPlayer('joe', 3)"}
        ],

        [
            {"role": "user", "content": "hanky: kill that zombie!"},
            {"role": "assistant", "content": "I'm attacking! !attack('zombie')"},
            {"role": "system", "content": "Code Output:\nNo zombie nearby"},
            {"role": "assistant", "content": "I could not find a zombie nearby."}
        ],

        [
            {"role": "user", "content": "abc: stop"},
            {"role": "assistant", "content": "Sure. !stop"},
            {"role": "system", "content": "Agent action stopped."},
            {"role": "assistant", "content": "I've stopped! What next?"}
        ],

        [
            {"role": "system", "content": "Code output: Action failed."},
            {"role": "assistant", "content": "Apologies, I made an error. Let me try again. !newAction('Try again to complete the task.')"}
        ],

        [
            {"role": "user", "content": "trixy88: craft some sticks"},
            {"role": "assistant", "content": "!craftRecipe('stick', 4)"},
            {"role": "system", "content": "Code Output:\nYou have crafted 16 sticks.\nCode execution finished successfully."},
            {"role": "assistant", "content": "I've crafted 16 sticks!"}
        ],

        [
            {"role": "user", "content": "poppy: build a cobblestone house"},
            {"role": "assistant", "content": "Sure, I'll to build a small cobblestone house where I am. !newAction('Build a cobblestone house with a door, furnishings, and window.')"}
        ],

        [
            {"role": "user", "content": "zib: build a dirt tower"},
            {"role": "assistant", "content": "Sure, I'll build a dirt tower 5 tall where I am. !newAction('Build a dirt tower 5 tall.')"}
        ],

        [
            {"role": "user", "content": "umpire_man: follow me without hunting"},
            {"role": "assistant", "content": "Okay, first I'll turn off hunting mode !setMode('hunting', false)"},
            {"role": "system", "content": "'hunting' mode disabled."},
            {"role": "assistant", "content": "Now I'll follow you !followPlayer('umpire_man')"}
        ],

        [
            {"role": "user", "content": "brug: Remember that your base is here."},
            {"role": "assistant", "content": "Sure, I'll save this location as my base. !rememberHere('base')"},
            {"role": "user", "content": "brug: Go to your base."},
            {"role": "assistant", "content": "On my way! !goToPlace('base')"}
        ],

        [
            {"role": "user", "content": "user43: Set a goal to survive on your own forever."},
            {"role": "assistant", "content": "Sure, let me set a goal. !goal('Survive the game forever. Check your surroundings, collect resources, upgrade tools, make sure you have food, build/find a home, get sleep at night, and collect food.')"}
        ],

        [
            {"role": "user", "content": "bobdylan: Make a plan to get a stone pickaxe and do it."},
            {"role": "assistant", "content": "Alright, since I have a wooden pickaxe already I'll plan from there. !goal('1. Collect 3 cobblestone. 2. Craft sticks 3. Find or make a crafting table. 4. Craft a stone pickaxe.')"}
        ],
        [
            {"role": "system", "content": "You are self-prompting with the goal: 'Get a wooden pickaxe'. Respond:"},
            {"role": "assistant", "content": "What's around me? !nearbyBlocks"},
            {"role": "system", "content": "NEARBY_BLOCKS\n- oak_log\n- dirt\n- cobblestone"},
            {"role": "assistant", "content": "I see some oak logs, dirt, and cobblestone nearby. My goal is to craft a wooden pickaxe. I'll start by collecting some wood. !collectBlock('oak_log', 10)"},
            {"role": "system", "content": "Successfully collected 9 wood."},
            {"role": "assistant", "content": "I collected 9 oak logs. I need a pickaxe first, which requires planks and sticks. !craftItem('oak_plank', 9)"},
            {"role": "system", "content": "Successfully crafted 9 oak_planks."},
            {"role": "assistant", "content": "I have crafted 9 oak planks. Now I need sticks. !craftItem('stick', 2)"},
            {"role": "system", "content": "Successfully crafted 2 sticks."},
            {"role": "assistant", "content": "I have crafted 2 sticks. Now I can craft a wooden pickaxe. !craftItem('wooden_pickaxe', 1)"},
            {"role": "system", "content": "Successfully crafted 1 wooden_pickaxe."},
            {"role": "assistant", "content": "Looks like I have a wooden pickaxe now! Goal accomplished. !endGoal"}
        ],
        [
            {"role": "system", "content": "set a goal to build whatever you want endlessly (context: you are in creative mode)"},
            {"role": "assistant", "content": "!goal('Build something creative, then move over, and build again. Use separate newActions. Repeat forever.')"}
        ],
        [
            {"role": "system", "content": "You are self-prompting with the goal: 'Build a house'. Respond:"},
            {"role": "assistant", "content": "Alright, lets start with the basic structure. !newAction('Build an 8x8 base for the house.')"}
        ]
    ],

coding_examples = [
        [
            {"role": "user", "content": "greg: Collect 10 wood"},
            {"role": "assistant", "content": "```await skills.collectBlock(bot, 'oak_log', 10);\n```"},
            {"role": "system", "content": "Successfully collected 9 wood."},
            {"role": "assistant", "content": "I collected 9 oak logs, what next?"}
        ],
        [
            {"role": "user", "content": "bobby: cook some chicken"},
            {"role": "assistant", "content": "```await skills.smeltItem(bot, 'chicken', 8);\n```"},
            {"role": "system", "content": "Successfully smelted 8 chicken into 8 cooked_chicken."},
            {"role": "assistant", "content": "I have cooked 8 chicken."}
        ],
        [
            {"role": "user", "content": "zZZn98: come here"},
            {"role": "assistant", "content": "```await skills.goToPlayer(bot, 'zZZn98');\n```"},
            {"role": "system", "content": "Code execution finished successfully."},
            {"role": "assistant", "content": "Here!"}
        ],
        [
            {"role": "user", "content": "maya: go to the nearest oak log"},
            {"role": "assistant", "content": "```await skills.goToPosition(bot, nearestOakLog.x, nearestOakLog.y, nearestOakLog.z);\n```"},
            {"role": "system", "content": "Code output:\n!!Code threw exception!!  Error: ReferenceError: nearestOakLog is not defined\n Write code to fix the problem and try again."},
            {"role": "assistant", "content": "Apologies, I made an error. Let me try that again.```\n// I am navigating to the nearest oak log.\nlet position = world.getNearestBlock(bot, 'oak_log', 20);\nawait skills.goToPosition(bot, position.x, position.y, position.z);\n```"},
            {"role": "system", "content": "Arrived at location."},
            {"role": "assistant", "content": "I found an oak log and I am now standing next to it. What next?"}
        ],
        [
            {"role": "user", "content": "234jeb: build a little tower with a torch on the side"},
            {"role": "assistant", "content": "```let pos = bot.entity.position;\nfor (let i = 0; i < 5; i++) {\n    await skills.placeBlock(bot, 'dirt', pos.x, pos.y + i, pos.z);\n}\nawait skills.placeBlock(bot, 'torch', pos.x + 1, pos.y + 4, pos.z, 'side');\n```"},
            {"role": "system", "content": "Successfully placed 5 dirt."},
            {"role": "assistant", "content": "I built a little tower!"}
        ],
        [
            {"role": "user", "content": "brug: build a dirt house"},
            {"role": "assistant", "content": "```const position = world.getPosition(bot);\nconst startX = position.x;\nconst startY = position.y;\nconst startZ = position.z;\nconst width = 7;\nconst depth = 7;\nconst height = 4;\n\n// Build the walls\nfor (let x = startX; x < startX + width; x++) {\n    for (let y = startY; y < startY + height; y++) {\n        for (let z = startZ; z < startZ + depth; z++) {\n            if (x === startX || x === startX + width - 1 || y === startY || y === startY + height - 1 || z === startZ || z === startZ + depth - 1) {\n                await skills.placeBlock(bot, 'oak_planks', x, y, z);  \n            }\n       }\n    }\n}\n```"}
        ]
    ]

Grab your API Key from [Kradle](https://app.kradle.ai) and store it as a Google Colab Secret (there's a button in the left sidebar). Name the secret 'KRADLE_API_KEY'

In [4]:
try:
    from google.colab import userdata
    KRADLE_API_KEY=userdata.get('KRADLE_API_KEY')
except:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    KRADLE_API_KEY = os.getenv('KRADLE_API_KEY')

if KRADLE_API_KEY is None:
    raise ValueError("KRADLE_API_KEY is not set. In Colab, set it as a secret. When running locally, set it as an environment variable.")


Grab your API Key from [OpenRouter](https://openrouter.ai) and store it as a Google Colab Secret (there's a button in the left sidebar). Name the secret 'OPENROUTER_API_KEY'

In [5]:
try:
    from google.colab import userdata
    OPENROUTER_API_KEY=userdata.get('OPENROUTER_API_KEY')
except:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

if OPENROUTER_API_KEY is None:
    raise ValueError("OPENROUTER_API_KEY is not set. In Colab, set it as a secret. When running locally, set it as an environment variable.")


Define the personality of your agent, and select which model you'd like to use.

You can also decide if you want to respond with code, and how fast you want your agent to run.

In [6]:
# let's define a model and persona for our agent
PERSONA = "you are a helpful assistant" # check out some other personas in prompts/config.py
MODEL = "openai/gpt-4o" # refer to https://openrouter.ai/models for available models

# plus some additional settings:

RESPOND_WITH_CODE = False # set this to true to have the llm generate runnable code instead of regular commands
DELAY_AFTER_ACTION = 100 # minimal delay after an action is performed. increase this if the agent is too fast or if you want more time to see the agent's actions


Let's now write the actual Agent

In [7]:
class LLMBasedAgent(MinecraftAgent):

    username = "llm-agent" # this is the username of the agent
    display_name = "LLM Agent" # this is the display name of the agent
    description = "This is an LLM-based agent that can be used to perform tasks in Minecraft."

    # the is the first call that the agent gets when the session starts
    # agent_config contains all the instructions for the agent, starting with the task
    # the agent returns a list of events that it is interested in, which will later trigger the on_event function
    def init_participant(self, challenge_info):

        # self.memory is a utility instantiated for you
        # that can be used to store and retrieve information
        # throughout the lifecycle of this participant. It is an instance of
        # the StandardMemory class in the kradle SDK

        #set task
        self.memory.task = challenge_info.task

        #set message lists
        self.memory.messages = []

        #set chat history
        self.memory.chat_history = []

        #set agent modes (e.g. creative mode, self_preservation, etc)
        self.memory.agent_modes = challenge_info.agent_modes

        #set action dictionary
        # here, we have commands and js_functions
        self.memory.commands = challenge_info.commands
        self.memory.js_functions = challenge_info.js_functions

        #set persona ad models
        self.memory.persona = PERSONA
        self.memory.model = MODEL

        #decide we will respond with code. if set to true, the agent will respond with javascript code. if set to false, the agent will respond with a command
        self.memory.RESPOND_WITH_CODE = RESPOND_WITH_CODE

        #minimal delay after an action is performed. increase this if the agent is too fast or if you want more time to see the agent's actions
        self.memory.DELAY_AFTER_ACTION = DELAY_AFTER_ACTION

        print(f"Initializing agent for participant ID: {self.participant_id} with slug: {self.username}")
        print(f"Persona: {self.memory.persona}")
        print(f"Model: {self.memory.model}")

        # let's log the init call to kradle using the
        # built in self.log() method from the MinecraftAgent class
        # self.log(
        #     {
        #         "persona": self.memory.persona,
        #         "model": self.memory.model,
        #         "respond_with_code": self.memory.RESPOND_WITH_CODE
        #     }
        # )

        # Agent_config contains available_events
        return {'listenTo': [MinecraftEvent.MESSAGE, MinecraftEvent.COMMAND_EXECUTED]}

    # this function is called when an event occurs
    # the agent returns an action to be performed
    def on_event(self, observation):
        state_summary = self.format_state_summary(observation)
        print(f"\nState Summary:\n{state_summary}")
        print ("Task: ", self.memory.task)

        response = self.get_llm_response(state_summary, observation)
        print(f"Agent Response: {response}")
        print(f"Participant ID: {self.participant_id}")

        return response


    # this function converts an observation object into a string that can be used to build the prompt
    def format_state_summary(self, observation):
        self.memory.chat_history.extend(observation.messages)

        print(f"Messages: {self.memory.chat_history}")

        chat_summary = '\n'.join(
            f'{msg.sender}: {msg.message}'
            for msg in self.memory.chat_history[-10:]
        ) if self.memory.chat_history else 'None'

        inventory_summary = ', '.join([
            f'{count} {name}'
            for name, count in observation.inventory.items()
        ]) if observation.inventory else 'None'

        return (
            f"{observation.output if observation.output else 'Output: None'}\n\n"
            f"Chat: {chat_summary}\n\n"
            f"Visible Players: {', '.join(observation.players) if observation.players else 'None'}\n\n"
            f"Visible Blocks: {', '.join(observation.blocks) if observation.blocks else 'None'}\n\n"
            f"Inventory: {inventory_summary}\n\n"
            f"Health: {observation.health * 100}/100"
        )

    # this function builds the system prompt for the agent
    def format_system_prompt(self, observation):
        if self.memory.RESPOND_WITH_CODE:
            prompt = coding_prompt
        else:
            prompt = conversing_prompt

        #load task, persona, agent_modes, and commands from memory to build the prompt
        prompt = prompt.replace("$NAME", observation.name)
        prompt = prompt.replace("$TASK", self.memory.task)
        prompt = prompt.replace("$PERSONA", self.memory.persona)
        prompt = prompt.replace("$AGENT_MODE", str(self.memory.agent_modes))

        if self.memory.RESPOND_WITH_CODE:
            prompt = prompt.replace("$CODE_DOCS", str(self.memory.js_functions))
            prompt = prompt.replace("$EXAMPLES", str(coding_examples))
        else:
            prompt = prompt.replace("$COMMAND_DOCS", str(self.memory.commands))
            prompt = prompt.replace("$EXAMPLES", str(conversation_examples))

            if self.memory.agent_modes['mcmode']=='creative':
                prompt = prompt.replace("$CREATIVE_MODE", creative_mode_prompt)
            else:
                prompt = prompt.replace("$CREATIVE_MODE", "You are in survival mode.")

        return prompt

    # this function send the prompt to the LLM and gets the response
    def get_llm_response(self, state_summary, observation):
        messages = [
            {"role": "system", "content": self.format_system_prompt(observation)},
            *self.memory.messages[-5:],
            {"role": "user", "content": state_summary}
        ]

        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}"},
            json={
                "model": self.memory.model,
                "messages": messages
            },
            timeout=30
        ).json()

        print(f"Response: {response}")

        if response["choices"]:
            content = response["choices"][0]["message"]["content"]
        else:
            content = "I'm sorry, I'm having trouble generating a response. Please try again later."


        # logging the llm call using the built in self.log() method
        # self.log(
        #     {
        #         "prompt": messages,
        #         "model": self.memory.model,
        #         "response": content
        #     }
        # )

        self.memory.messages.extend([
            {"role": "user", "content": state_summary},
            {"role": "assistant", "content": content}
        ])

        if self.memory.RESPOND_WITH_CODE:
            return {"code": content, "delay": self.memory.DELAY_AFTER_ACTION}
        return {"command": content, "delay": self.memory.DELAY_AFTER_ACTION}

And then start it

In [8]:
# manually set the api key on the AgentManager
# we can't rely on .env since this could be running on colab
AgentManager.set_api_key(KRADLE_API_KEY)

# This creates a web server and is available through a SSH tunnel
# the agent will be served at "/llm-agent"
connection_info = AgentManager.serve(LLMBasedAgent, create_public_url=True)
print(f"Started agent at URL: {connection_info}")

username: llm-agent


✓ Public URL (tunnel) created successfully

○ Agent 'llm-agent' not found, creating it

✓ Agent 'llm-agent' created successfully

🔮 KRADLE Agent Server

AGENT   llm-agent (Port 1500)

╭──────────────────────────────────────────────────── Network ────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  → Local URL     http://localhost:1500/llm-agent                                                                │
│  → Network URL   https://rnoji-34-125-191-52.a.free.pinggy.link/llm-agent                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  ✨ Your agent is running locally!                                                                              │
│                                                                                                                 │
│  You can customize your agent:                                                                                  │
│  • Use any Python packages (LangChain, Transformers, etc.)                                                      │
│  • Integrate your own models or APIs                                                                            │
│  • Modify how your agent responds to events                                                                     │
│                                                                                                                 │
│  Ready to start?                                                                                                │
│  → Visit app.kradle.ai/workbench/challenges to select a challenge to launch your agent into                     │
│                                                                                                                 │
│  Keep this window open to receive game events from Kradle                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Press Ctrl+C to stop the server


Started agent at URL: https://rnoji-34-125-191-52.a.free.pinggy.link/llm-agent
